In [5]:
from os import walk
from joblib import Parallel, delayed
import librosa
import numpy as np
import multiprocessing
import json
from deep_audio import Directory, Audio, JSON

In [6]:
num_cores = multiprocessing.cpu_count()

sampling_rate = 16000

path = f'audios/{sampling_rate}'

f = Directory.filenames(path)

data = {
    "mapping": [],
    "mfcc": [],
    "labels": []
}

In [7]:
def process_directory(dir, index):
    signal, sr = Audio.read(f'{path}/{dir}', sr=sampling_rate, normalize=True)

    signal = np.array(signal)

    signal = signal[:len(signal) - len(signal) % (sr * 5)]

    segments = len(signal) // (sr * 5)

    m = {
        "mfcc": [],
        "labels": [index] * segments
    }

    for i in range(segments):
        start_sample = sr * i * 5
        finish_sample = start_sample + (sr * 5)

        sample = signal[start_sample:finish_sample]

        # sample = Audio.normalize(sample)

        # mfcc = Audio.mfcc(sample, rate=sr)
        mfcc = librosa.feature.mfcc(sample, sr=sr, n_mfcc=13, hop_length=512, n_fft=2048, lifter=22)

        # mfcc = mfcc.T

        m['mfcc'].append(mfcc.tolist())

    print(f'{dir} -> segments: {segments}')
    return m

In [8]:
def object_mfcc_to_json(m):
    data['mapping'] = [file.replace('.wav', '') for file in f]

    for i in m:
        data['mfcc'].extend(i['mfcc'])
        data['labels'].extend(i['labels'])

    JSON.create_json_file(f'processed/mfcc/mfcc_{sampling_rate}.json', data)


In [9]:
if __name__ == '__main__':
    m = Parallel(n_jobs=num_cores, verbose=len(f), temp_folder='./tmp/')(
        delayed(process_directory)(i, j) for j, i in enumerate(f) if i is not None)
    object_mfcc_to_json(m)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    3.7s
[Parallel(n_jobs=2)]: Done   2 tasks      | elapsed:    3.8s
[Parallel(n_jobs=2)]: Done   3 tasks      | elapsed:    5.2s
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    5.6s
[Parallel(n_jobs=2)]: Done   5 tasks      | elapsed:    6.7s
[Parallel(n_jobs=2)]: Done   6 tasks      | elapsed:    7.0s
[Parallel(n_jobs=2)]: Done   7 tasks      | elapsed:    7.9s
[Parallel(n_jobs=2)]: Done   8 tasks      | elapsed:    8.4s
[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed:    9.6s
[Parallel(n_jobs=2)]: Done  10 tasks      | elapsed:    9.7s
[Parallel(n_jobs=2)]: Done  11 tasks      | elapsed:   11.0s
[Parallel(n_jobs=2)]: Done  12 tasks      | elapsed:   11.4s
[Parallel(n_jobs=2)]: Done  13 tasks      | elapsed:   12.7s
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:   13.1s
[Parallel(n_jobs=2)]: Done  15 tasks      | elapsed:   14.3s
[Parallel(